In [1]:
import itertools as it
from collections import defaultdict
import numpy as np

def hash_function(pair, num_buckets):
    i, j = pair
    hash_value = ((i * num_buckets) + j) % num_buckets
    return hash_value

In [2]:
with open("./retail.csv") as file:
    baskets = file.readlines()

transactions = [[int(item) for item in basket.strip().split(' ')] for basket in baskets]

NUM_BUCKETS = 10000000
SUPPORT = round(0.01*len(transactions))
frequent_items = []

In [3]:
# Pass 1: counting singletons and hashing pairs to buckets
    # each pair will be sorted in order to maintain consistent hashing for out of order pairs
C1 = defaultdict(int)
buckets = np.zeros(10000000)

for basket in transactions:
    for i in basket:
        C1[i] += 1
    
    for p in it.combinations(basket,2):
        buckets[hash_function(sorted(p),NUM_BUCKETS)] += 1

In [4]:
# Pruning C1 to create L1 for singletons
# converting buckets to a bit-vector

items = tuple(C1)
for i in items:
    if C1[i] < SUPPORT:
        del C1[i]
# L1 is mapping of frequent singletons frequent singletons
L1 = C1
for i in L1:
    frequent_items.append(list(i))

# converting buckets to a bit vector
for x in range(len(buckets)):
    if buckets[x] < SUPPORT:
        buckets[x] = 0
    else:
        buckets[x] = 1

In [5]:
# Pass 2: Generating C2 using L1 and bit vector to reduce candidate pair list
C2 = defaultdict(int)
for basket in transactions:
    for p in it.combinations(basket,2):
        p = tuple(sorted(p))
        bucket_num = hash_function(p, NUM_BUCKETS)
        if( (p[0] in L1) and (p[1] in L1) and (buckets[bucket_num] == 1)):
            C2[p] += 1

In [6]:
# pruning infrequent item pairs
pairs = tuple(C2)
for p in pairs:
    if C2[p] < SUPPORT:
        del C2[p]
L2 = C2
for p in L2:
    frequent_items.append(list(p))
